# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output1'
TRAIT = 'Lupus-(Systemic-Lupus-Erythematosus'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [25]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

utils.py has been loaded


['TCGA_Colon_and_Rectal_Cancer_(COADREAD)']

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Lupus-(Systemic-Lupus-Erythematosus"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE112943',
 'GSE153459',
 'GSE180393',
 'GSE180394',
 'GSE184989',
 'GSE193442',
 'GSE200306',
 'GSE224705',
 'GSE72326']

Repeat the below steps for all the accession numbers

In [4]:
cohort = accession_num = 'GSE224705'
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Lupus-(Systemic-Lupus-Erythematosus\\GSE224705\\GSE224705_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Lupus-(Systemic-Lupus-Erythematosus\\GSE224705\\GSE224705_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [5]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Defining molecular and cellular landscape behind non-response to Mycophenolate Mofetil in Systemic Lupus Erythematosus"
!Series_summary	"Lupus nephritis (LN) is one of the more severe systemic lupus erythematosus manifestations with the potential of developing into end stage kidney disease. Mycophenolate mofetil (MMF) is widely used for both induction and maintenance therapy for LN, but the one year complete renal response ranges from 30-40% in most trials. Reasons for non-response are still unknown.  Thus, anticipating lack of drug efficacy in a patient could lead to early introduction of advanced therapies.  A longitudinal cohort comprising gene-expression and clinical data of 97 MMF responder and 28 non-responder samples was retrospectively analyzed. Response to MMF was defined over time according to whether the protein/creatinine ratio in urine was reduced and maintained below 500 mg/g with the use of the treatment. Differential gene expression and functional analysi

In [6]:
clinical_data.head()

,!Sample_geo_accession,GSM7029843,GSM7029844,GSM7029845,GSM7029846,GSM7029847,GSM7029848,GSM7029849,GSM7029850,GSM7029851,...,GSM7029958,GSM7029959,GSM7029960,GSM7029961,GSM7029962,GSM7029963,GSM7029964,GSM7029965,GSM7029966,GSM7029967
0,!Sample_characteristics_ch1,days_from_first_visit: 0,days_from_first_visit: 52,days_from_first_visit: 82,days_from_first_visit: 124,days_from_first_visit: 143,days_from_first_visit: 0,days_from_first_visit: 52,days_from_first_visit: 0,days_from_first_visit: 35,...,days_from_first_visit: 0,days_from_first_visit: 93,days_from_first_visit: 0,days_from_first_visit: 82,days_from_first_visit: 0,days_from_first_visit: 89,days_from_first_visit: 0,days_from_first_visit: 89,days_from_first_visit: 0,days_from_first_visit: 110
1,!Sample_characteristics_ch1,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,...,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Male,Sex: Male,Sex: Female,Sex: Female,Sex: Female,Sex: Female
2,!Sample_characteristics_ch1,age: 18.71,age: 18.71,age: 18.71,age: 18.71,age: 18.71,age: 17.66,age: 17.66,age: 25.41,age: 25.41,...,age: 28.97,age: 28.97,age: 56.22,age: 56.22,age: 35.15,age: 35.15,age: 15.24,age: 15.24,age: 40.11,age: 40.11
3,!Sample_characteristics_ch1,mmf_response: NO,mmf_response: NO,mmf_response: NO,mmf_response: NO,mmf_response: NO,mmf_response: NO,mmf_response: NO,mmf_response: YES,mmf_response: YES,...,mmf_response: YES,mmf_response: YES,mmf_response: YES,mmf_response: YES,mmf_response: YES,mmf_response: YES,mmf_response: YES,mmf_response: YES,mmf_response: YES,mmf_response: YES
4,!Sample_characteristics_ch1,sledai: 4,sledai: 0,sledai: 8,sledai: 0,sledai: 0,sledai: 10,sledai: 6,sledai: 4,sledai: 0,...,sledai: 0,sledai: 0,sledai: 2,sledai: 8,sledai: 1,sledai: 1,sledai: 2,sledai: 0,sledai: 4,sledai: 4


In [7]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['days_from_first_visit: 0',
  'days_from_first_visit: 52',
  'days_from_first_visit: 82',
  'days_from_first_visit: 124',
  'days_from_first_visit: 143',
  'days_from_first_visit: 35',
  'days_from_first_visit: 126',
  'days_from_first_visit: 219',
  'days_from_first_visit: 295',
  'days_from_first_visit: 384',
  'days_from_first_visit: 629',
  'days_from_first_visit: 91',
  'days_from_first_visit: 85',
  'days_from_first_visit: 68',
  'days_from_first_visit: 109',
  'days_from_first_visit: 38',
  'days_from_first_visit: 157',
  'days_from_first_visit: 96',
  'days_from_first_visit: 97',
  'days_from_first_visit: 268',
  'days_from_first_visit: 79',
  'days_from_first_visit: 170',
  'days_from_first_visit: 435',
  'days_from_first_visit: 560',
  'days_from_first_visit: 163',
  'days_from_first_visit: 27',
  'days_from_first_visit: 58',
  'days_from_first_visit: 284',
  'days_from_first_visit: 382',
  'days_from_first_visit: 478'],
 1: ['Sex: Female', 'Sex: Male'],
 2: ['age: 18.71

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [8]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Lupus-(Systemic-Lupus-Erythematosus\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Lupus-(Systemic-Lupus-Erythematosus\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique va

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [9]:
is_gene_availabe = True
trait_row = 3
age_row = 2
gender_row = 1

trait_type = "binary"

In [10]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [11]:
# def convert_trait(tissue_type):
#     """
#     Convert tissue type to epilepsy presence (binary).
#     Assuming epilepsy presence for 'Hippocampus' tissue.
#     """
#     if tissue_type == 'ln class: Healthy control':
#         return 0  # Epilepsy present
#     else:
#         return 1  # Epilepsy not present

# def convert_age1(age_string):
#     """
#     Convert age string to a numerical value.
#     """
#     if age_string.startswith('age (yrs): '):
#         try:
#             return int(age_string.split('age (yrs): ')[1].rstrip(''))  # Extracting numeric part
#         except ValueError:
#             return None  # In case of parsing error
#     else:
#         return None  # If age string does not follow expected format
# def convert_age(age_string):
#     """
#     将年龄字符串转换为数字值。
#     """
#     if isinstance(age_string, str) and age_string.startswith('age (yrs): '):
#         try:
#             return int(age_string.split('age (yrs): ')[1].rstrip(''))  # 提取数值部分
#         except ValueError:
#             return None  # 处理解析错误的情况
#     else:
#         return None  # 如果年龄字符串不是字符串或不符合预期格式

# def convert_gender(gender_string):
#     """
#     Convert gender string to a binary representation.
#     Female -> 0, Male -> 1, Unknown -> None
#     """
#     if gender_string == 'Sex: F':
#         return 0
#     elif gender_string == 'Sex: M':
#         return 1
#     else:
#         return None  # In case of unknown gender

# # Example usage
# example_tissue = 'tissue: Hippocampus'
# example_age = 'age: 23y'
# example_gender = 'gender: Male'

# converted_trait = convert_trait(example_tissue)
# converted_age = convert_age(example_age)
# converted_gender = convert_gender(example_gender)

# converted_trait, converted_age, converted_gender
# def convert_trait(value):
#     if value == 'YES':
#         return True
#     elif value == 'NO':
#         return False
#     else:
#         return None

# def convert_age(value):
#     try:
#         return float(value)
#     except ValueError:
#         return None

# def convert_gender(value):
#     if value == 'Male':
#         return 'Male'
#     elif value == 'Female':
#         return 'Female'
#     else:
# Function to convert Lupus trait value to binary data type
# Function to convert Lupus trait value to binary data type
# def convert_trait(value):
#     unique_values = ['disease state: DLE', 'disease state: SCLE', 'disease state: ACLE', 'disease state: NN']
#     if value in unique_values:
#         return 1  # Presence of the disease state
#     else:
#         return 0  # Absence or unknown

# # Placeholder function for converting age to continuous data type
# def convert_age(value):
#     # Placeholder code until specific information about age data is available
#     return None  # Replace None with the actual conversion code

# # Placeholder function for converting gender to binary data type
# def convert_gender(value):
#     # Placeholder code until specific information about gender data is available
#     return None  # Replace None with the actual conversion code

def convert_trait(value):
    if value == 'mmf_response: YES':
        return True
    elif value == 'mmf_response: NO':
        return False
    else:
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except ValueError:
        return None

def convert_gender(value):
    if value == 'Sex: Female':
        return 'Female'
    elif value == 'Sex: Male':
        return 'Male'
    else:
        return None









In [12]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM7029843,GSM7029844,GSM7029845,GSM7029846,GSM7029847,GSM7029848,GSM7029849,GSM7029850,GSM7029851,GSM7029852,...,GSM7029958,GSM7029959,GSM7029960,GSM7029961,GSM7029962,GSM7029963,GSM7029964,GSM7029965,GSM7029966,GSM7029967
Lupus-(Systemic-Lupus-Erythematosus,False,False,False,False,False,False,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
Age,18.71,18.71,18.71,18.71,18.71,17.66,17.66,25.41,25.41,25.41,...,28.97,28.97,56.22,56.22,35.15,35.15,15.24,15.24,40.11,40.11
Gender,Female,Female,Female,Female,Female,Female,Female,Female,Female,Female,...,Female,Female,Female,Female,Male,Male,Female,Female,Female,Female


### Genetic data preprocessing and final filtering

In [13]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM7029843,GSM7029844,GSM7029845,GSM7029846,GSM7029847,GSM7029848,GSM7029849,GSM7029850,GSM7029851,GSM7029852,...,GSM7029958,GSM7029959,GSM7029960,GSM7029961,GSM7029962,GSM7029963,GSM7029964,GSM7029965,GSM7029966,GSM7029967
ID,,,,,,,,,,,,,,,,,,,,,
1007_PM_s_at,4.231006,4.698801,3.910390,3.580336,3.070567,4.482996,4.533454,3.565169,3.592472,3.732844,...,4.091762,3.972350,4.096282,4.096387,4.488520,4.199651,4.090172,3.965488,4.371665,4.658340
1053_PM_at,6.468179,6.169779,6.581117,6.092602,6.173526,5.751040,6.284164,6.645881,5.516079,6.439264,...,5.998896,5.737181,5.746002,5.713781,5.375176,6.364288,5.993029,5.867990,6.101444,6.408576
117_PM_at,10.030950,9.236338,9.809446,9.241966,8.533765,9.164631,9.969305,9.368355,9.653980,9.513935,...,8.463895,9.829381,8.510235,9.794175,9.117856,9.185171,9.324569,8.762964,8.873801,9.318778
121_PM_at,4.773703,4.512929,4.561124,4.580938,4.450634,4.733456,4.521983,3.494578,4.748343,4.934529,...,5.009729,4.842848,4.444645,4.335791,4.371740,4.757004,4.045731,4.711662,4.309774,4.071375
1255_PM_g_at,1.859223,2.181506,2.388049,2.213596,2.131014,1.998884,1.992263,2.124469,2.190746,1.954034,...,2.224990,1.980419,2.306177,1.889213,2.270244,2.262065,2.235232,2.084425,2.071246,2.027748


In [14]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1007_PM_s_at',
 '1053_PM_at',
 '117_PM_at',
 '121_PM_at',
 '1255_PM_g_at',
 '1294_PM_at',
 '1316_PM_at',
 '1320_PM_at',
 '1405_PM_i_at',
 '1431_PM_at',
 '1438_PM_at',
 '1487_PM_at',
 '1494_PM_f_at',
 '1552256_PM_a_at',
 '1552257_PM_a_at',
 '1552258_PM_at',
 '1552261_PM_at',
 '1552263_PM_at',
 '1552264_PM_a_at',
 '1552266_PM_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [15]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at', '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at', '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at', '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at', '1552264_PM_a_at', '1552266_PM_at']\n"

If not required, jump directly to the gene normalization step

In [16]:
requires_gene_mapping = False

In [17]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [103]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

In [104]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [18]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [19]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [20]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 125 samples.


In [21]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lupus-(Systemic-Lupus-Erythematosus', the least common label is 'False' with 28 occurrences. This represents 22.40% of the dataset.
The distribution of the feature 'Lupus-(Systemic-Lupus-Erythematosus' in this dataset is fine.

Quartiles for 'Age':
  25%: 18.71
  50% (Median): 25.41
  75%: 28.97
Min: 12.39
Max: 60.05
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is 'Male' with 14 occurrences. This represents 11.20% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [22]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [23]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [58]:
from utils import *

In [59]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

In [60]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [ ]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

,Psoriasis,Age,Gender,-0.001001579,-0.010425045,-0.045320336,-0.046967808,-0.069891929,-0.081080626,-0.093186199,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
0,0.0,56.0,1.0,3.293947,3.344254,3.049427,3.489313,3.713489,3.182711,2.885457,...,11.631593,9.914236,5.934949,4.756848,10.265351,10.109506,4.033606,11.065690,8.733339,9.681242
1,0.0,20.0,0.0,2.295976,3.490590,3.402046,3.960615,2.295976,3.209781,3.380016,...,11.685883,9.888024,5.981049,3.573693,10.808423,10.135307,3.854406,10.578108,8.207179,9.830063
2,0.0,51.0,0.0,2.878075,3.642838,3.281445,3.630912,2.878075,3.658581,3.466199,...,11.823756,9.977897,5.965869,4.439257,10.669074,10.243314,4.174246,10.824677,8.554421,10.367084
3,0.0,37.0,1.0,3.567006,3.393262,3.477423,3.000214,3.190889,3.070480,2.543753,...,11.871873,9.948733,6.094711,4.479720,10.832092,10.253960,3.735004,10.505996,8.786537,9.695719
4,0.0,61.0,1.0,4.034010,3.607132,2.822304,3.712049,3.126172,3.017512,2.949723,...,11.931741,10.017533,6.085654,3.807143,10.252109,10.396369,4.112947,10.797622,8.826644,8.821104


In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

The cross-validation accuracy is 56.84% ± 4.59%


In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0025
Found 547 significant genes associated with the trait 'Psoriasis' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
4.430015695     0.087750           0.001231
4.478193647     0.087750           0.001231
4.478966158     0.087750           0.001231
 4.48687579     0.087750           0.001231
4.491851159     0.087750           0.001231
4.492453853     0.087750           0.001231
4.494270678     0.087750           0.001231
4.504245863     0.087750           0.001231
4.505190735     0.087750           0.001231
4.505235806     0.087750           0.001231
4.506891502     0.087750           0.001231
4.510669146     0.087750           0.001231
4.476613942     0.087750           0.001231
 4.51198966     0.087750           0.001231
4.513528153     0.087750           0.001231
4.516930407     0.087750           0.001231
4.806825082     0.087750           0.001

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)